https://www.dominodatalab.com/blog/getting-started-with-k-means-clustering-in-python

In [ ]:
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
# import some data to play with
data = pd.read_csv('kmeans.csv', header='infer')
le = preprocessing.LabelEncoder()
le.fit(data['class'])
data['class']=le.transform(data['class'])

#data1 = data.drop(columns=['class', "mfcc_2_mean", "mfcc_3_mean",  "mfcc_4_mean", "mfcc_5_mean", "mfcc_6_mean", "mfcc_7_mean", "mfcc_8_mean","mfcc_9_mean","mfcc_10_mean","mfcc_11_mean","mfcc_12_mean",	"mfcc_13_mean"])
data1 = data.sample(frac=0.05, replace=True, random_state=1)
data = data.sample(frac=0.05, replace=True, random_state=1)

X = data1
y = data['class']
#names = iris.feature_names

In [ ]:

#Put data onto the same standard scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(data1)
data1=scaler.fit_transform(data1)
data1

In [ ]:
plt.scatter(data1[1:,0],data1[1:,1:])
plt.xlabel('ZCR_MEAN')
plt.ylabel('MFCC_1_MEAN')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42) 
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
kmeans.labels_

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix=confusion_matrix(y, kmeans.labels_) 

fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', 
                ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
kmeans.cluster_centers_


In [ ]:
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

https://notebook.community/ml4a/ml4a-guides/notebooks/audio-tsne

In [ ]:
%pip install librosa==0.7.0


In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import fnmatch
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import json
from sklearn.cluster import KMeans
import pandas as pd

scan some directory of audio files and collect all their paths into a single list

In [2]:
path = 'D:\\PhD-data\\denoising\\post\\Wambiana'

files = []
for root, dirnames, filenames in os.walk(path):
    for filename in fnmatch.filter(filenames, '*.wav'):
        files.append(os.path.join(root, filename))

print("found %d .wav files in %s"%(len(files),path))

found 2237 .wav files in D:\PhD-data\denoising\post\Wambiana


In [ ]:
# import librosa
# import numpy as np
# import pandas as pd

# # Create an empty list to store the extracted features
# features = []

# # Loop through all file paths
# for file_path in files:
#     # Load the audio file
#     y, sr = librosa.load(file_path)
    
#     # Extract the MFCC features and their first and second order derivatives
#     mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, lifter=0, n_fft=2048, hop_length=512)
#     mfcc_delta = librosa.feature.delta(mfcc)
#     mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    
#     # Concatenate the features and their derivatives into a single array
#     features_row = np.concatenate((mfcc, mfcc_delta, mfcc_delta2))
    
#     # Add the filename as the final column
#     file_name = file_path.split('/')[-1]  # Extract the filename from the file path
#     features_row = np.append(features_row, file_name)
    
#     # Add the row of features to the list of features
#     features.append(features_row)

# # Convert the list of features to a DataFrame and save as a .csv file
# df = pd.DataFrame(features)
# df.to_csv('output.csv', header=False, index=False)


function which extracts a feature vector from an audio file

The feature extraction will calculate the first 13 mel-frequency cepstral coefficients of the audio file, as well as their first- and second-order derivatives, and concatenate them into a single 39-element feature vector. The feature vector is also standardized so that each feature has equal variance.

In [3]:
def get_features(y, sr):
    y = y[0:sr]  # analyze just first second
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    log_S = librosa.amplitude_to_db(S, ref=np.max)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    delta_mfcc = librosa.feature.delta(mfcc, mode='nearest')
    delta2_mfcc = librosa.feature.delta(mfcc, order=2, mode='nearest')
    feature_vector = np.concatenate((np.mean(mfcc,1), np.mean(delta_mfcc,1), np.mean(delta2_mfcc,1)))
    feature_vector = (feature_vector-np.mean(feature_vector)) / np.std(feature_vector)
    return feature_vector

Now we will iterate through all the files, and get their feature vectors, placing them into a new list feature_vectors. We also make a new array sound_paths to index the feature vectors to the correct paths

In [4]:
import librosa
feature_vectors = []
sound_paths = []
for i,f in enumerate(files):
    if i % 100 == 0:
        print("get %d of %d = %s"%(i+1, len(files), f))
    y, sr = librosa.load(f)
    if len(y) < 2:
        print("error loading %s" % f)
        continue
    feat = get_features(y, sr)
    feature_vectors.append(feat)
    sound_paths.append(f)
        
print("calculated %d feature vectors"%len(feature_vectors))

get 1 of 2237 = D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA100_1.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA100_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA101_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA102_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA103_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA104_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA105_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA106_14.wav
get 101 of 2237 = D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA107_11.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA107_14.wav
error loading D:\PhD-data\denoising\post\Wambiana\anthrophony\audio_segment_dryA108_1

In [5]:
from sklearn import preprocessing
import pandas as pd

x = feature_vectors
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
feature_vectors = pd.DataFrame(x_scaled)
print(feature_vectors)
df = pd.DataFrame(feature_vectors)
paths = pd.DataFrame(sound_paths)
df = pd.concat([df, paths], ignore_index=True, sort=False, axis=1)
df.to_csv('39-features-wambiana-denoised-spectral.csv', index=False)

            0         1         2         3         4         5         6   \
0     0.082471  0.454871  0.796889  0.506489  0.627111  0.696046  0.539483   
1     0.171055  0.443470  0.693785  0.653337  0.713298  0.512147  0.649591   
2     0.083777  0.471251  0.792655  0.505085  0.659196  0.666593  0.571229   
3     0.088650  0.460130  0.798181  0.507931  0.645169  0.692779  0.554799   
4     0.092463  0.481952  0.809069  0.513986  0.663942  0.657764  0.568602   
...        ...       ...       ...       ...       ...       ...       ...   
2072  0.171812  0.682472  0.607981  0.384563  0.409399  0.324260  0.629749   
2073  0.230628  0.737007  0.615823  0.425049  0.407289  0.355502  0.619014   
2074  0.216930  0.728404  0.605230  0.426588  0.439569  0.323748  0.593072   
2075  0.197193  0.705339  0.621347  0.452494  0.428704  0.332111  0.561994   
2076  0.182078  0.680637  0.595624  0.457195  0.419943  0.311313  0.570475   

            7         8         9   ...        29        30    

Now we can run t-SNE over the feature vectors to get a 2-dimensional embedding of our audio files. We use scikit-learn's TSNE function, and additionally normalize the results so that they are between 0 and 1.

In [ ]:
import pandas as pd
feature_vectors = pd.read_csv("39-features-tarcutta-sound.csv")
feature_vectors

In [ ]:
import pandas as pd

# Drop rows containing missing values
feature_vectors = feature_vectors.dropna()
#feature_vectors.to_csv('39-features-esc50-cleaned.csv', index=False)
feature_vectors = feature_vectors.drop(columns=["class"])
feature_vectors = feature_vectors.drop(columns=["path"])
feature_vectors = feature_vectors.iloc[:, 0:13]
feature_vectors


In [ ]:
model = TSNE(n_components=2, learning_rate=150, perplexity=50, verbose=2, angle=0.1).fit_transform(feature_vectors)


Let's plot our t-SNE points. We can use matplotlib to quickly scatter them and see their distribution.

In [ ]:
x_axis=model[:,0]
y_axis=model[:,1]

plt.figure(figsize = (10,10))
plt.scatter(x_axis, y_axis)
plt.show()

We see our t-SNE plot of our audio files, but it's not particularly interesting! Since we are dealing with audio files, there's no easy way to compare neighboring audio samples to each other. We can use some other, more interactive environment to view the results of the t-SNE. One way we can do this is by saving the results to a JSON file which stores the filepaths and t-SNE assignments of all the audio files. We can then load this JSON file in another environment.

In any case, to save the t-SNE to a JSON file, we first normalize the coordinates to between 0 and 1 and save them, along with the full filepaths.

In [ ]:
from pyAudioProcessing import utils


tsne_path = "example-audio-tSNE.json"

x_norm = (x_axis - np.min(x_axis)) / (np.max(x_axis) - np.min(x_axis))
y_norm = (y_axis - np.min(y_axis)) / (np.max(y_axis) - np.min(y_axis))

data = [{"path":os.path.abspath(f), "point":[x, y]} for f, x, y in zip(sound_paths, x_norm, y_norm)]
# with open(tsne_path, 'w') as outfile:
utils.write_to_json(tsne_path, str(data))
    # json.dump(data, outfile, cls=)

print("saved %s to disk!" % tsne_path)

In [ ]:
km = KMeans(n_clusters=4, init='k-means++', random_state=42)
y_km = km.fit_predict(model)
labels = km.labels_

In [ ]:
model[y_km == 0, 0]

In [ ]:
# plot the 3 clusters
plt.figure(figsize=(20, 20))
plt.scatter(
    model[y_km == 0, 0], model[y_km == 0, 1],
    s=50, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    model[y_km == 1, 0], model[y_km == 1, 1],
    s=50, c='orange',
    marker='o', edgecolor='black',
    label='cluster 2'
)

plt.scatter(
    model[y_km == 2, 0], model[y_km == 2, 1],
    s=50, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 3'
)

plt.scatter(
    model[y_km == 3, 0], model[y_km == 3, 1],
    s=50, c='blue',
    marker='v', edgecolor='black',
    label='cluster 3'
)

# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1)
plt.title('Clusters by t-SNE Components (with Dimension Reduction)', fontsize=20)
plt.xlabel("Component 2", fontsize=18)
plt.ylabel("Component 1", fontsize=18)
plt.grid()
plt.show()

In [ ]:
kmeans_data_old = pd.read_csv('kmeans_results.csv')

#encoding class values
kmeans_data_old['class'] = kmeans_data_old['class'].str.replace('other', str(0))
kmeans_data_old['class'] = kmeans_data_old['class'].str.replace('biophony', str(1))
kmeans_data_old['class'] = kmeans_data_old['class'].str.replace('geophony', str(2))
kmeans_data_old['class'] = kmeans_data_old['class'].str.replace('anthrophony', str(3))

y_km = kmeans_data_old['class']
y_km.drop(y_km.tail(1).index, inplace=True)
y_km = y_km.values
y_km

In [ ]:
%pip install seaborn

In [ ]:
import seaborn as sns

x_axis=model[:,0]
y_axis=model[:,1]

plt.figure(figsize=(16,10))
ax=sns.scatterplot(
    x=x_axis, y=y_axis,
    hue=y_km,
    legend="full",
    alpha=0.3,
    palette=['green','orange','dodgerblue','red']
)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['Anthrophony', 'Biophony', 'Geophony', 'Other'])
ax.set(title="TSNE Plot with Ground Truth Overlayed", ylabel="TSNE Component 1", xlabel="TSNE Component 2")

TODO: GET STATISTICS FOR ALL OF THE ABOVE GRAPHS
TODO: CREATE ADDITIONAL GRAPH (USING SAME TSNE RESULTS) WITHOUT DIMENSION REDUCTION

In [ ]:
kmeans_data_new = pd.read_csv('kmeans-nodr.csv', header='infer')
kmeans_data_new.drop(kmeans_data_new.tail(1).index,inplace=True) # drop last n rows

y_km = kmeans_data_new["Cluster"]

In [ ]:
import seaborn as sns

x_axis=model[:,0]
y_axis=model[:,1]

plt.figure(figsize=(16,10))
ax=sns.scatterplot(
    x=x_axis, y=y_axis,
    hue=y_km,
    legend="full",
    alpha=0.3,
    palette=['green','orange','dodgerblue','red']
)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['Geophony', 'Anthrophony', 'Biophony', 'Other'])
ax.set(title="Kmeans Clustering Results without Dimension Reduction", ylabel="TSNE Component 1", xlabel="TSNE Component 2")

In [ ]:
kmeans_data_old = pd.read_csv('kmeans_results.csv', header='infer')

kmeans_data_old.drop(kmeans_data_old.tail(1).index,inplace=True) # drop last n rows
y_km = kmeans_data_old["kmeans_class"]



In [ ]:
import seaborn as sns

x_axis=model[:,0]
y_axis=model[:,1]

plt.figure(figsize=(16,10))
ax=sns.scatterplot(
    x=x_axis, y=y_axis,
    hue=y_km,
    legend="full",
    alpha=0.3,
    palette=['green','orange','dodgerblue','red']
)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['Anthrophony', 'Other', 'Biophony', 'Geophony'])
ax.set(title="Kmeans Clustering Results with Dimension Reduction", ylabel="TSNE Component 1", xlabel="TSNE Component 2")

You will however be able to calculate statistics like within group variance and between group variance. The goal is to have low within group variance and high between group variance

In [ ]:
print(labels)

TODO: append results to k-means and use metrics to evaluate results against ground truth data

In [ ]:
df = pd.read_csv('kmeans.csv')
print(df)


In [ ]:
kmeans_df = pd.DataFrame(labels, columns=["kmeans_class"])

df = pd.concat([df, kmeans_df], axis=1)
print(df)
df.to_csv("kmeans_results.csv", index=False)

Rand Index - similarity of the two clustering assignments
Biophony = 0
Other = 1
Anthrophony = 2
Geophony = 3

In [ ]:
#kmeans_df['kmeans_class'].value_counts()

In [ ]:
from sklearn import metrics
kmeans_data = pd.read_csv('kmeans_results.csv', header='infer')

labels_pred = kmeans_data['kmeans_class']


#encoding class values
kmeans_data['class'] = kmeans_data['class'].str.replace('other', str(1))
kmeans_data['class'] = kmeans_data['class'].str.replace('biophony', str(0))
kmeans_data['class'] = kmeans_data['class'].str.replace('geophony', str(3))
kmeans_data['class'] = kmeans_data['class'].str.replace('anthrophony', str(2))

#print(kmeans_data)

labels_true = kmeans_data['class']

In [ ]:
metrics.rand_score(labels_true, labels_pred)

The Rand index does not ensure to obtain a value close to 0.0 for a random labelling. The adjusted Rand index corrects for chance and will give such a baseline.

In [ ]:
metrics.adjusted_rand_score(labels_true, labels_pred)

Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information (NMI) and Adjusted Mutual Information (AMI). NMI is often used in the literature, while AMI was proposed more recently and is normalized against chance:

In [ ]:
metrics.adjusted_mutual_info_score(labels_true, labels_pred)

homogeneity: each cluster contains only members of a single class.

completeness: all members of a given class are assigned to the same cluster.

In [ ]:
metrics.homogeneity_score(labels_true, labels_pred)

In [ ]:
metrics.completeness_score(labels_true, labels_pred)

Their harmonic mean called V-measure is computed by v_measure_score:

In [ ]:
metrics.v_measure_score(labels_true, labels_pred)

The Fowlkes-Mallows score FMI is defined as the geometric mean of the pairwise precision and recall. A high value indicates a good similarity between two clusters

In [ ]:
metrics.fowlkes_mallows_score(labels_true, labels_pred)